In [21]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

This imports PyTorch, alongside datasets, transforms, and DataLoader, which will help get the images to the CNN.
Torch.nn is for the neural network layers, and torch.nn.functional is for activation functions.

In [22]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])



This will resize and make the images into a tensor for our CNN

In [23]:
train_dataset = datasets.ImageFolder(root="MLDL_Data_Face-1/train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = datasets.ImageFolder(root="MLDL_Data_Face-1/val", transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


Here, we create our datasets and loaders using the imports. We have our training data set composed of subjects 1 and 2, for all three classes. We want to transform them as defined above, then make a batch size (32 is adjustable). Shuffling the training set helps with randomness, while shuffling the validation set would be detrimental.

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 3)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = SimpleCNN()
print(model)

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=32768, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [26]:
num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/30], Loss: 0.8035
Epoch [2/30], Loss: 0.3031
Epoch [3/30], Loss: 0.1702
Epoch [4/30], Loss: 0.1317
Epoch [5/30], Loss: 0.1040
Epoch [6/30], Loss: 0.0813
Epoch [7/30], Loss: 0.0747
Epoch [8/30], Loss: 0.0624
Epoch [9/30], Loss: 0.0608
Epoch [10/30], Loss: 0.0456
Epoch [11/30], Loss: 0.0495
Epoch [12/30], Loss: 0.0444
Epoch [13/30], Loss: 0.0438
Epoch [14/30], Loss: 0.0328
Epoch [15/30], Loss: 0.0288
Epoch [16/30], Loss: 0.0417
Epoch [17/30], Loss: 0.0229
Epoch [18/30], Loss: 0.0244
Epoch [19/30], Loss: 0.0273
Epoch [20/30], Loss: 0.0251
Epoch [21/30], Loss: 0.0178
Epoch [22/30], Loss: 0.0248
Epoch [23/30], Loss: 0.0170
Epoch [24/30], Loss: 0.0167
Epoch [25/30], Loss: 0.0199
Epoch [26/30], Loss: 0.0153
Epoch [27/30], Loss: 0.0158
Epoch [28/30], Loss: 0.0184
Epoch [29/30], Loss: 0.0147
Epoch [30/30], Loss: 0.0139


In [27]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Validation Accuracy after Epoch [{epoch+1}/{num_epochs}]: {accuracy * 100:.2f}%")
print(f"Validation Accuracy: {accuracy * 100:.2f}%")
print(correct)
print(total)


Validation Accuracy after Epoch [30/30]: 8.14%
Validation Accuracy: 8.14%
142
1745


https://arxiv.org/pdf/1902.03524


https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=ffe8cc49b14ede3807b91b4fa5217daf8515fdb2


https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8053088casa_token=eNqo9STTYNwAAAAA:y28zM5DEUi6EWK7AbP07CDo6VEQjvsM-k_S5tI2XrFo9GwI_wS8D_gKDZKHCpU-J6ftcr00WCA&tag=1


Trial 1 - LR: .001, Epochs: 5, Accuracy: 2.06% - Unoptimized

Trial 2 - LR: .001, Epochs: 5, Accuracy: 2.64% - Added normalization and augmentation

Trial 3 - LR: .001, Epochs: 20, Accuracy: 31.86% - Upped the Epochs

Trial 3 - LR: .001, Epochs: 30, Accuracy: % - Upped the Epochs